In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.device(0)
    torch.cuda.manual_seed_all(777)
    
    gpu = torch.cuda.get_device_properties(0)
    print("GPU Name:{}\nGPU Capability: (Major: {}, Minor: {})".format(
        gpu.name, gpu.major, gpu.minor
    ))

GPU Name:GeForce RTX 2080 SUPER
GPU Capability: (Major: 7, Minor: 5)


In [3]:
learning_rate = .001
training_epochs = 15
batch_size = 100
keep_prob = .7

In [4]:
from torch.utils.data import DataLoader

mnist_train = dsets.MNIST(root="./MNIST_data/", train=True, transform=transforms.ToTensor(), download=True)
mnist_test  = dsets.MNIST(root="./MNIST_data/", train=False, transform=transforms.ToTensor(), download=True)

data_loader = DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [5]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(784, 512, bias=True)
        self.linear2 = nn.Linear(512, 512, bias=True)
        self.linear3 = nn.Linear(512, 512, bias=True)
        self.linear4 = nn.Linear(512, 512, bias=True)
        self.linear5 = nn.Linear(512, 10, bias=True)
        self.relu    = nn.ReLU()
        self.dropout = nn.Dropout(p=1-keep_prob)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear4(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear5(x)
        return x
    
    def weight_init(self):
        layers = list(model.children())
        
        for layer in layers:
            try:
                nn.init.xavier_uniform_(layer.weight)
            except Exception as e:
                pass

#### model.train() & model.eval()
 - <strong>"Sets the module in train \& evaluation mode."</strong>

In [6]:
model = Model().to(device)

In [7]:
model.weight_init()

In [8]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    clear_output(wait=True)
    print("Epoch: {:4d} Cost = {:.9f}".format(epoch+1, avg_cost))
print('Learning finished')

Epoch:   15 Cost = 0.040947560
Learning finished


In [21]:
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    # Test the model using test sets
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9776999950408936
Label:  8
Prediction:  3


C:\Users\admin\workspace\Pytorch\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\admin\workspace\Pytorch\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
